### Problem 332. Reconstruct Itinerary
Problem Statement:
- You are given a list of airline tickets where tickets[i] = [fromi, toi] represent the departure and the arrival airports of one flight. Reconstruct the itinerary in order and return it.
- All of the tickets belong to a man who departs from "JFK", thus, the itinerary must begin with "JFK". If there are multiple valid itineraries, you should return the itinerary that has the smallest lexical order when read as a single string.
- For example, the itinerary ["JFK", "LGA"] has a smaller lexical order than ["JFK", "LGB"].
You may assume all tickets form at least one valid itinerary. You must use all the tickets once and only once.

Solution Strategy:
- use dept first search in the context of graph problem
- must sort dest in reversed order so that we explore dest which have lower lexilogical first
- must use while when explore node

In [ ]:
from collections import defaultdict
class Solution:
    def findItinerary(self, tickets: List[List[str]]) -> List[str]:
        #init graph
        graph = defaultdict(list)

        #build graph
        #sorted dest so that we go to lower lexical order first
        for src, dest in sorted(tickets, reverse= True):
            graph[src].append(dest)
        itinerary = []
        @cache
        def dfs(src):
            #go to dest, must use while in here
            while graph[src]:
                dfs(graph[src].pop())
            #append src
            itinerary.append(src)
                #init itinerary
        
        dfs("JFK")
        return itinerary[::-1]
            

### Problem 1584. Min Cost to Connect All Points
Problem Statement
- You are given an array points representing integer coordinates of some points on a 2D-plane, where points[i] = [xi, yi].
- The cost of connecting two points [xi, yi] and [xj, yj] is the manhattan distance between them: |xi - xj| + |yi - yj|, where |val| denotes the absolute value of val.
- Return the minimum cost to make all points connected. All points are connected if there is exactly one simple path between any two points.

Solution Strategy:
- use list with index for visited
- use cache to memorize dist
- only explore node which have lower dist can in current cache


In [ ]:
import heapq
class Solution:
    def minCostConnectPoints(self, points: List[List[int]]) -> int:
        n = len(points)
        min_cost = 0
        #check nodes have been visited
        visited = [False] * n
        #keep track of min cost and vertex
        pq = [(0, 0)]  # (cost, vertex)
        
        cache = {0: 0} #{v: cost}

        while pq:
            cost, u = heapq.heappop(pq)

            if visited[u]:
                continue

            visited[u] = True
            min_cost += cost
            #traverse through vertex by layer
            for v in range(n):
                if not visited[v]:
                    dist = abs(points[u][0] - points[v][0]) + abs(points[u][1] - points[v][1])
                    #only explore node which have lower dist in cache
                    if dist < cache.get(v, float('inf')):
                        cache[v] = dist #keep track of lower dist cache
                        heapq.heappush(pq, (dist, v))

        return min_cost        

### Problem 743. Network Delay Time
Problem Statement:
- You are given a network of n nodes, labeled from 1 to n. You are also given times, a list of travel times as directed edges times[i] = (ui, vi, wi), where ui is the source node, vi is the target node, and wi is the time it takes for a signal to travel from source to target.
- We will send a signal from a given node k. Return the minimum time it takes for all the n nodes to receive the signal. If it is impossible for all the n nodes to receive the signal, return -1.

Solution Strategy:


In [ ]:
from collections import defaultdict
from heapq import heappop, heappush
class Solution:
    def networkDelayTime(self, times: List[List[int]], n: int, k: int) -> int:        

        graph = defaultdict(list)
        for u, v, w in times:
            graph[u].append((w, v ))

        pq = [(0, k)] #(w, v)
        visited = set()

        while pq:
            travel_time, u = heappop(pq)

            #skip visited nodes
            if u in visited:
                continue
            #mark node have been visited
            visited.add(u)
            #return when reach all nodes
            if len(visited) == n:
                return travel_time
            for w, v in graph[u]:
                if v not in visited:
                    #carry total delay time
                    heappush(pq, (travel_time+w, v))
        return -1

### Problem 778. Swim in Rising Water
Problem Statement:
- You are given an n x n integer matrix grid where each value grid[i][j] represents the elevation at that point (i, j).
- The rain starts to fall. At time t, the depth of the water everywhere is t. You can swim from a square to another 4-directionally adjacent square if and only if the elevation of both squares individually are at most t. You can swim infinite distances in zero time. Of course, you must stay within the boundaries of the grid during your swim.
- Return the least time until you can reach the bottom right square (n - 1, n - 1) if you start at the top left square (0, 0).

Solution Strategy:
- use breadth first search with carry value


In [ ]:
class Solution:
    def swimInWater(self, grid: List[List[int]]) -> int:
        
        n = len(grid)

        pq = [(grid[0][0], 0, 0)] #(time, r, c)
        visited = set() #v
        directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
        while pq:
            min_time, r, c = heappop(pq)
            
            if (r, c) in visited:
                continue
            visited.add((r, c))


            if r == n-1 and c == n-1:
                return min_time
            #move to neighbors
            for dx, dy in directions:
                r_dx, c_dy = r + dx, c + dy
                if 0 <= r_dx < n and 0 <= c_dy < n and (r_dx, c_dy) not in visited:
                    #time to reach v
                    heappush(pq, (max(grid[r_dx][c_dy], min_time), r_dx, c_dy))
        
            

### Problem 787. Cheapest Flights Within K Stops
Problem Statement:
- There are n cities connected by some number of flights. You are given an array flights where flights[i] = [fromi, toi, pricei] indicates that there is a flight from city fromi to city toi with cost pricei.
- You are also given three integers src, dst, and k, return the cheapest price from src to dst with at most k stops. If there is no such route, return -1.

Solution Strategy:
- use modified dijkstra's algorithm


In [ ]:
#Modified Dijkstra's Algorithm
from collections import defaultdict
from heapq import *
class Solution:
    def findCheapestPrice(self, n, flights, src, dst, k):
        #compute min_cost

        #build graph
        graph = defaultdict(list)
        for u,v, cost in flights:
            # graph.setdefault(u,[])
            graph[u].append((v,cost))

        cache, pq = [float('inf')] * n ,[(0, src, 0)]
        
        while pq:
            steps,u,cost = heappop(pq)
            # condition continue explore the graph
            if steps<=k and u in graph:
                for v,w in graph[u]:
                    #only choose dest which have lower cost
                    if w + cost < cache[v]:
                        cache[v] = w + cost #keep track of lowest cost
                        heappush(pq,(steps+1, v, w+cost))
        return cache[dst] if cache[dst] != float('inf') else -1

### Problem Alient Dictionary
Problem Statement:
- There is a new alien language which uses the latin alphabet. However, the order among letters are unknown to you. You receive a list of non-empty words from the dictionary, where words are sorted lexicographically by the rules of this new language. Derive the order of letters in this language.

- Example 1:

- Input:
- [
  "wrt",
  "wrf",
  "er",
  "ett",
  "rftt"
]

- Output: "wertf"
- Example 2:

- Input:
- [
  "z",
  "x"
]

- Output: "zx"
- Example 3:

- Input:
- [
  "z",
  "x",
  "z"
] 

- Output: "" 

- Explanation: The order is invalid, so return "".
- Note:

    - You may assume all letters are in lowercase.
    - You may assume that if a is a prefix of b, then a must appear before b in the given dictionary.
    - If the order is invalid, return an empty string.
    - There may be multiple valid order of letters, return any one of them is fine.

In [ ]:
class Solution(object):
    def alienOrder(self, words):
        # Find ancestors of each node by DFS.
        n = len(words)
        adj = {c: set() for w in words for c in w}

        for i in range(n-1):
            w1, w2 = words[i], words[i+1]
            minLen = min(len(w1), len(w2))
            #check invalid order
            if len(w1) > len(w2) and w1[:minLen] == w2[:minLen]:
                return ""
            for j in range(minLen):
                if w1[j] != w2[j]:
                    adj[w1[j]].add(w2[j])
                    break
        
        visited = {}
        res = []

        def dfs(c):
            #condition to detect cicle
            if c in visited:
                return visited[c]
            #makr node have been in the path
            visited[c] = True

            for nei in adj[c]:
                if dfs(nei):
                    return True
            #mark as not visited for other paths to be explore
            visited[c] = False
            res.append(c) #because of dfs add leaf node first

        for c in adj:
            if dfs(c):
                return ""
        #reverse to get correct order
        return "".join(res[::-1])